In [1]:
import lxml.html as lx
import lxml.html as lx
import numpy as np
import pandas as pd
import requests
import requests_cache
import time

requests_cache.install_cache("cache")

In [2]:
print("此版本仅供旅人栈桥-Soma丸子亲友使用")
print("属于测试版本，BUG巨多")

此版本仅供旅人栈桥-Soma丸子亲友使用
属于测试版本，BUG巨多


In [3]:
print('读取"成品列表.xlsx"')
product=pd.read_excel("成品列表.xlsx")

读取"成品列表.xlsx"


In [9]:
while 1:
    print("请输入是哪个大区（数字）,比如2区就输入2")
    a = input("")
    if a == '1':
        dcName = '陆行鸟'
        break
    if a == '2':
        dcName = '莫古力'
        break   
    if a == '3':
        dcName = '猫小胖'
        break  
print('选择了'+dcName+'服务器')

请输入是哪个大区（数字）,比如2区就输入2
2


In [ ]:
while 1:
    url='http://universalis.diemoe.net/api/'
    print('输入服务器名字，可以直接输入"1"为旅人栈桥，2为梦羽宝境，其他服务器需要手动输入名字')
    a = input("")
    if a == "1":
        worldID = '旅人栈桥'
        break
    if a == "2":
        worldID = '梦羽宝境'
        break
    response = requests.get(url+a+"/2")
    b = response.json()
    if b['listings'] != []:
        worldID = a
        break
    print('嗯？你输入的这个服务器怎么连火之碎晶都没有人在卖的，或者是universalis炸了,重新输入吧')
print('选择了'+worldID+'服务器')

输入服务器名字，可以直接输入"1"为旅人栈桥，2为梦羽宝境，其他服务器需要手动输入名字
哈哈
嗯？你输入的这个服务器怎么连火之碎晶都没有人在卖的，或者是universalis炸了,重新输入吧
输入服务器名字，可以直接输入"1"为旅人栈桥，2为梦羽宝境，其他服务器需要手动输入名字


In [7]:
#创建ID字典
ID = {}
ID_product = {}

In [8]:
url = "https://ff14.huijiwiki.com/wiki/物品:"
for i in range(len(product)):
    time.sleep(0.1)
    try:
        response = requests.get(url+product.iloc[i,0])
    except:
        print('有点问题哈，检查一下excel文件里面，黄色区域之外的内容有没有删除')
        print('直接关闭窗口吧，相当于遇到BUG了，我还没解决')
        os.system("pause")
    try:
        response.raise_for_status()
        doc = response.text
        html = lx.fromstring(doc)
        first = html.xpath("//td/ul/li/ul/li/div/span[1]")#读取直接材料名字
        first_num = html.xpath("//td/ul/li/ul/li/div/span[2]")#读取直接材料数量
        locals()["fir"+str(i)] = [x.text_content() for x in first]#记录一级材料名字
        locals()["fir_num"+str(i)] = [int(x.text_content().split('×',)[1]) for x in first_num]#记录一级材料对应数量
        ID_product[product.iloc[i,0]]=html.xpath("//*[@class='ff14-hardcore']")[0].text_content().split("：")[1]#记录成品的ID
    except:
        print("wiki无法找到目标成品:"+product.iloc[i,0])
        locals()["fir"+str(i)]  = []
        locals()["fir_num"+str(i)] = []

TypeError: can only concatenate str (not "float") to str

In [ ]:
fir = []
for i in range(len(product)):
    fir += locals()["fir"+str(i)]
fir = list(set(fir))
fir.sort()

In [ ]:
url = "https://ff14.huijiwiki.com/wiki/物品:"
for i in range(len(fir)):
    time.sleep(0.1)
    try:
        response = requests.get(url+fir[i])
        response.raise_for_status()
        doc = response.text
        html = lx.fromstring(doc)
        second = html.xpath("//td/ul/li/ul/li/div/span[1]")
        second_num = html.xpath("//td/ul/li/ul/li/div/span[2]")
        locals()["sec"+str(i)] = [x.text_content() for x in second]#记录需要的二级材料名字
        locals()["sec_num"+str(i)] = [int(x.text_content().split('×',)[1]) for x in second_num]#记录需要的二级材料对应数量
        ID[fir[i]]=html.xpath("//*[@class='ff14-hardcore']")[0].text_content().split("：")[1]#记录一级材料的ID
    except:
        print("wiki有关页面404了:"+fir[i])
        locals()["sec"+str(i)] = []
        locals()["sec_num"+str(i)] = []

In [ ]:
sec = []
for i in range(len(fir)):
    sec += locals()["sec"+str(i)]
sec = list(set(sec))
sec.sort()

In [ ]:
url = "https://ff14.huijiwiki.com/wiki/物品:"
for i in range(len(sec)):
    time.sleep(0.1)
    try:
        response = requests.get(url+sec[i])
        response.raise_for_status()
        doc = response.text
        html = lx.fromstring(doc)
        ID[sec[i]]=html.xpath("//*[@class='ff14-hardcore']")[0].text_content().split("：")[1]#记录二级材料的ID
    except:
        print("wiki有关页面404了:"+sec[i])

In [ ]:
url = "http://universalis.diemoe.net/api/"+worldID+"/"
url2 = "http://universalis.diemoe.net/api/"+dcName+"/"
product_price=pd.DataFrame(columns=(worldID+'HQ最低价格',worldID+'NQ最低价格',dcName+'HQ最低价格','HQ低价所在服务器',dcName+'NQ最低价格','NQ低价所在服务器'))
for i in ID_product:
    time.sleep(0.1)
    #看本服价格
    response = requests.get(url+ ID_product[i])
    a = response.json()
    #hq最低价格
    if [x['pricePerUnit'] for x in a['listings'] if x['hq']==True]:
        hqprice = [x['pricePerUnit'] for x in a['listings'] if x['hq']==True][0]
    else:
        hqprice = '无'
    #nq最低价格
    if [x['pricePerUnit'] for x in a['listings'] if x['hq']==False]:
        nqprice = [x['pricePerUnit'] for x in a['listings'] if x['hq']==False][0]
    else:
        nqprice = '无'
    #看本区价格
    response = requests.get(url2+ ID_product[i])
    a = response.json()
    #本区hq最低价格
    if [x['pricePerUnit'] for x in a['listings'] if x['hq']==True]:
        dchqprice = [x['pricePerUnit'] for x in a['listings'] if x['hq']==True][0]
        dchqworld = [x['worldName'] for x in a['listings'] if x['hq']==True][0]
    else:
        dchqprice = '无'
        dchqworld = '无'
    #本区nq最低价格
    if [x['pricePerUnit'] for x in a['listings'] if x['hq']==False]:
        dcnqprice = [x['pricePerUnit'] for x in a['listings'] if x['hq']==False][0]
        dcnqworld = [x['worldName'] for x in a['listings'] if x['hq']==False][0]
    else:
        dcnqprice = '无'
        dcnqworld = '无'
    
    product_price.loc[i]=[hqprice,nqprice,dchqprice,dchqworld,dcnqprice,dcnqworld]

In [ ]:
url = "http://universalis.diemoe.net/api/"+worldID+"/"
url2 = "http://universalis.diemoe.net/api/"+dcName+"/"
material_price=pd.DataFrame(columns=(worldID+'最低价格',dcName+'最低价格','低价所在服务器'))
for i in ID:
    time.sleep(0.1)
    #看本服价格
    response = requests.get(url+ ID[i])
    a = response.json()
    #最低价格
    if [x['pricePerUnit'] for x in a['listings']]:
        price = [x['pricePerUnit'] for x in a['listings']][0]
    else:
        price = '无'
    #看本区价格
    response = requests.get(url2+ ID[i])
    a = response.json()
    #本区最低价格
    if [x['pricePerUnit'] for x in a['listings']]:
        dcprice = [x['pricePerUnit'] for x in a['listings']][0]
        dcworld = [x['worldName'] for x in a['listings']][0]
    else:
        dcprice = '无'
        dcworld = '无'

    material_price.loc[i]=[price,dcprice,dcworld]

In [ ]:
#现在需要取得是输入材料名，给出最低价格
#这个部分没有考虑市场上最低价格的材料够不够数量

In [ ]:
first_cost=pd.DataFrame(columns=(worldID+'最低板子价格',
                                 '只用'+worldID+'材料来制作',
                                 dcName+'最低板子价格',
                                 "板子最低所在服务器",
                                 dcName+"最便宜材料来制作"))
for i in range(len(fir)):
    url = "https://ff14.huijiwiki.com/wiki/物品:"
    try:
        response = requests.get(url+fir[i])
        response.raise_for_status()
        doc = response.text
        html = lx.fromstring(doc)
        #一次制作做多少个
        test = html.xpath("//td/ul/li/div//*[@class='item-number']")
        craft_num = [int(x.text_content().split('×',)[1]) for x in test]
        craft_num += [1]
        #需要的材料以及数量
        test = html.xpath("//td/ul/li/ul/li/div/span[1]")
        test_num = html.xpath("//td/ul/li/ul/li/div/span[2]")
        cost = [x.text_content() for x in test]
        cost_num = [int(x.text_content().split('×',)[1]) for x in test_num]
        
        #计算成本
        test = []
        test2 = []
        for j in cost:
            test += [int(material_price.loc[j,worldID+'最低价格'])]
            test2 += [int(material_price.loc[j,dcName+"最低价格"])]
        crafted_cost = sum(np.multiply(cost_num,test))/craft_num[0]
        crafted_cost2 = sum(np.multiply(cost_num,test2))/craft_num[0]
        first_cost.loc[fir[i]]=[material_price.loc[fir[i],worldID+'最低价格'],
                                crafted_cost,
                                material_price.loc[fir[i],dcName+'最低价格'],
                                material_price.loc[fir[i],"低价所在服务器"],
                                crafted_cost2]
    except:
        first_cost.loc[fir[i]]=[9999999,9999999,9999999,"无法取得物品ID网页404",9999999]

In [ ]:
#用最便宜的方案制作一级材料，看是要直接买，还是自己做,生成一级材料价格表
material = first_cost.copy()
del material['只用'+worldID+'材料来制作']
del material[dcName+"最便宜材料来制作"]
del material['板子最低所在服务器']
material.columns=[worldID,dcName]
for i in range(len(material)):
    if first_cost.iloc[i,1]!= 0:#排除无法制作的商品
        if first_cost.iloc[i,1]<first_cost.iloc[i,0]:#判断是否用制作价格替换市场价格
            material.loc[first_cost.index[i],worldID]=int(first_cost.iloc[i,1])
        if first_cost.iloc[i,4]<first_cost.iloc[i,2]:#判断是否用制作价格替换市场价格
            material.loc[first_cost.index[i],dcName]=int(first_cost.iloc[i,4])

In [ ]:
product_cost=pd.DataFrame(columns=(worldID+'HQ最低价格',
                                   worldID+'NQ最低价格',
                                   '只用'+worldID+"材料制作",
                                    dcName+'HQ最低价格',
                                    'HQ低价所在服务器',
                                    dcName+'NQ最低价格',
                                    'NQ低价所在服务器',
                                   dcName+"最低成本价"))
for i in range(len(product)):
    url = "https://ff14.huijiwiki.com/wiki/物品:"
    try:
        response = requests.get(url+product.iloc[i,0])
        response.raise_for_status()
        doc = response.text
        html = lx.fromstring(doc)
        #一次制作做多少个
        test = html.xpath("//td/ul/li/div//*[@class='item-number']")
        craft_num = [int(x.text_content().split('×',)[1]) for x in test]
        craft_num += [1]
        #需要的材料以及数量
        test = html.xpath("//td/ul/li/ul/li/div/span[1]")
        test_num = html.xpath("//td/ul/li/ul/li/div/span[2]")
        cost = [x.text_content() for x in test]
        cost_num = [int(x.text_content().split('×',)[1]) for x in test_num]
        
        #计算成本
        test = []
        test2 = []
        for j in cost:
            test += [int(material.loc[j,worldID])]
            test2 += [int(material.loc[j,dcName])]
        crafted_cost = sum(np.multiply(cost_num,test))/craft_num[0]
        crafted_cost2 = sum(np.multiply(cost_num,test2))/craft_num[0]
        product_cost.loc[product.iloc[i,0]]=[product_price.loc[product.iloc[i,0],worldID+'HQ最低价格'],
                                             product_price.loc[product.iloc[i,0],worldID+'NQ最低价格'],
                                             crafted_cost,
                                             product_price.loc[product.iloc[i,0],dcName+'HQ最低价格'],                                             
                                             product_price.loc[product.iloc[i,0],'HQ低价所在服务器'],
                                             product_price.loc[product.iloc[i,0],dcName+'NQ最低价格'],
                                             product_price.loc[product.iloc[i,0],'NQ低价所在服务器'],
                                             crafted_cost2]
    except:
        product_cost.loc[product.iloc[i,0]]=["X","X","X","无法取得物品ID网页404","X","X","X","X"]

In [ ]:
pd.DataFrame(product_cost).to_csv(r'./成品价格对比.csv',index=True,sep=',',encoding='utf_8_sig')
pd.DataFrame(first_cost).to_csv(r'./一级材料价格对比.csv',index=True,sep=',',encoding='utf_8_sig')
pd.DataFrame(material_price).to_csv(r'./基础材料价格一栏.csv',index=True,sep=',',encoding='utf_8_sig')
print('搞定')
os.system("pause")